In [1]:

from __future__ import division
from keras.models import load_model
import cv2
import numpy as np

from random import choice
from math import sqrt
import random as rnd

Using TensorFlow backend.


In [2]:
REV_CLASS_MAP = {
    0: "rock",
    1: "paper",
    2: "scissors",
    3: "none"
}



def mapper(val):
    return REV_CLASS_MAP[val]

In [3]:
playerscore = 0
compscore = 0
tiescore = 0
def calculate_winner(move1, move2):
    global playerscore
    global compscore
    global tiescore
    if move1 == move2:
        tiescore = tiescore + 1
        return "Tie"

    if move1 == "rock":
        if move2 == "scissors":
            playerscore = playerscore + 1
            return "User"
        if move2 == "paper":
            compscore = compscore + 1
            return "Computer"

    if move1 == "paper":
        if move2 == "rock":
            playerscore = playerscore + 1
            return "User"
        if move2 == "scissors":
            compscore = compscore + 1
            return "Computer"

    if move1 == "scissors":
        if move2 == "paper":
            playerscore = playerscore + 1
            return "User"
        if move2 == "rock":
            compscore = compscore + 1
            return "Computer"

In [4]:
model = load_model("rock-paper-scissors-model.h5")

cap = cv2.VideoCapture(0)

cap.set(3, 1920)
cap.set(4, 1080)

prev_move = None

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [5]:
RPS_count = { '000' : 3, '001' : 3, '002' : 3, '010' : 3, '011' : 3, '012' : 3, '020' : 3, '021' : 3, '022' : 3, '100' : 3, '101' : 3, '102' : 3, '110' : 3, '111' : 3, '112' : 3, '120' : 3, '121' : 3, '122' : 3, '200' : 3, '201' : 3, '202' : 3, '210' : 3, '211' : 3, '212' : 3, '220' : 3, '221' : 3, '222' : 3 }

RPS_disp  = {'0' : 'rock', '1' : 'paper', '2' : 'scissor'}



last2 = '33'


while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # rectangle for user to play
    cv2.rectangle(frame, (100, 100), (500, 500), (255, 255, 255), 2)
    # rectangle for computer to play
    cv2.rectangle(frame, (800, 100), (1200, 500), (255, 255, 255), 2)

    # extract the region of image within the user rectangle
    roi = frame[100:500, 100:500]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (227, 227))

    # predict the move made
    pred = model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "none":
            
            
            if user_move_name == "rock":
                x='0'
                
            if user_move_name == "paper":
                x='1'
                
            if user_move_name == "scissors":
                x='2'
                
       
            
            
            if(last2[0] == '3'):
                computer_move_name = choice(['rock', 'paper', 'scissors'])
                
            else:
                r_count = RPS_count[last2 + '0']
                p_count = RPS_count[last2 + '1']
                s_count = RPS_count[last2 + '2']
                
                print('r_count ', r_count)
                print('p_count ', p_count)
                print('s_count ', s_count)
                
       

                tot_count = r_count + p_count + s_count
                
            
                print('tot_count ',tot_count)
        

                q_dist = [ r_count/tot_count, p_count/tot_count, 1- (r_count/tot_count) - (p_count/tot_count) ]
            
                print('q_dist ', q_dist)

                result = [ max(q_dist[2]-q_dist[1],0), max(q_dist[0]-q_dist[2],0), max(q_dist[1]-q_dist[0],0) ]
                
                print('result ',result)
                
                resultnorm = sqrt(result[0]*result[0] + result[1]*result[1] + result[2]*result[2])
                result = [result[0]/resultnorm, result[1]/resultnorm, 1 - result[0]/resultnorm - result[1]/resultnorm]
                
                
                
                
                print('resultnorm ',resultnorm)
                
                print('result ',result)

                y = rnd.uniform(0,1)
                
                
                print('y ',y)

                if y <= result[0]:
                    computer_move_name = "rock"
                elif y <= result[0] + result[1]:
                    computer_move_name = "paper"
                else:
                    computer_move_name = "scissors"
                
                
                print('computer_move_name ',computer_move_name)
              

                
                #update dictionary
                RPS_count[last2+x] += 1
                
                
                print('RPS_count ',RPS_count)
        
                    
                
            last2 = last2[1] + x
            
            print('last2 ',last2)

           
            
            
            winner = calculate_winner(user_move_name, computer_move_name)
        else:
            computer_move_name = "none"
            winner = "Waiting..."
    prev_move = user_move_name

    # display the information
    font = cv2.FONT_HERSHEY_SIMPLEX   
    
    cv2.putText(frame, "Player: " + str(playerscore),
                (100, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, "Tie: " + str(tiescore),
                (450, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.putText(frame, "Computer: " + str(compscore),
                (800, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    
    
    
    cv2.putText(frame, "Your Move: " + user_move_name,
                (100, 70), font, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (800, 70), font, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "Winner: " + str(winner),
                (400, 600), font, 2, (0, 0, 255), 4, cv2.LINE_AA)

    if computer_move_name != "none":
        icon = cv2.imread(
            "images/{}.png".format(computer_move_name))
        icon = cv2.resize(icon, (400, 400))
        frame[100:500, 800:1200] = icon

    cv2.imshow("Rock Paper Scissors", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

last2  30
last2  01
r_count  3
p_count  3
s_count  3
tot_count  9
q_dist  [0.3333333333333333, 0.3333333333333333, 0.3333333333333334]
result  [1.1102230246251565e-16, 0, 0.0]
resultnorm  1.1102230246251565e-16
result  [1.0, 0.0, 0.0]
y  0.6584690011404104
computer_move_name  rock
RPS_count  {'000': 3, '001': 3, '002': 3, '010': 3, '011': 3, '012': 4, '020': 3, '021': 3, '022': 3, '100': 3, '101': 3, '102': 3, '110': 3, '111': 3, '112': 3, '120': 3, '121': 3, '122': 3, '200': 3, '201': 3, '202': 3, '210': 3, '211': 3, '212': 3, '220': 3, '221': 3, '222': 3}
last2  12
r_count  3
p_count  3
s_count  3
tot_count  9
q_dist  [0.3333333333333333, 0.3333333333333333, 0.3333333333333334]
result  [1.1102230246251565e-16, 0, 0.0]
resultnorm  1.1102230246251565e-16
result  [1.0, 0.0, 0.0]
y  0.95163729098106
computer_move_name  rock
RPS_count  {'000': 3, '001': 3, '002': 3, '010': 3, '011': 3, '012': 4, '020': 3, '021': 3, '022': 3, '100': 3, '101': 3, '102': 3, '110': 3, '111': 3, '112': 3, '120

r_count  3
p_count  6
s_count  3
tot_count  12
q_dist  [0.25, 0.5, 0.25]
result  [0, 0.0, 0.25]
resultnorm  0.25
result  [0.0, 0.0, 1.0]
y  0.30832527759875816
computer_move_name  scissors
RPS_count  {'000': 3, '001': 3, '002': 3, '010': 3, '011': 3, '012': 7, '020': 3, '021': 3, '022': 3, '100': 3, '101': 3, '102': 3, '110': 3, '111': 3, '112': 3, '120': 7, '121': 5, '122': 3, '200': 3, '201': 7, '202': 3, '210': 3, '211': 3, '212': 5, '220': 3, '221': 3, '222': 3}
last2  01
r_count  3
p_count  3
s_count  7
tot_count  13
q_dist  [0.23076923076923078, 0.23076923076923078, 0.5384615384615383]
result  [0.30769230769230754, 0, 0.0]
resultnorm  0.30769230769230754
result  [1.0, 0.0, 0.0]
y  0.7291259275251237
computer_move_name  rock
RPS_count  {'000': 3, '001': 3, '002': 3, '010': 4, '011': 3, '012': 7, '020': 3, '021': 3, '022': 3, '100': 3, '101': 3, '102': 3, '110': 3, '111': 3, '112': 3, '120': 7, '121': 5, '122': 3, '200': 3, '201': 7, '202': 3, '210': 3, '211': 3, '212': 5, '220': 3

r_count  7
p_count  4
s_count  3
tot_count  14
q_dist  [0.5, 0.2857142857142857, 0.2142857142857143]
result  [0, 0.2857142857142857, 0]
resultnorm  0.2857142857142857
result  [0.0, 1.0, 0.0]
y  0.04874686555043284
computer_move_name  paper
RPS_count  {'000': 3, '001': 3, '002': 3, '010': 5, '011': 3, '012': 7, '020': 7, '021': 5, '022': 3, '100': 3, '101': 3, '102': 6, '110': 3, '111': 3, '112': 3, '120': 7, '121': 5, '122': 3, '200': 3, '201': 8, '202': 6, '210': 4, '211': 3, '212': 5, '220': 3, '221': 3, '222': 3}
last2  21
r_count  4
p_count  3
s_count  5
tot_count  12
q_dist  [0.3333333333333333, 0.25, 0.41666666666666674]
result  [0.16666666666666674, 0, 0]
resultnorm  0.16666666666666674
result  [1.0, 0.0, 0.0]
y  0.5490745191160927
computer_move_name  rock
RPS_count  {'000': 3, '001': 3, '002': 3, '010': 5, '011': 3, '012': 7, '020': 7, '021': 5, '022': 3, '100': 3, '101': 3, '102': 6, '110': 3, '111': 3, '112': 3, '120': 7, '121': 5, '122': 3, '200': 3, '201': 8, '202': 6, '210